# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [1]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [342]:
STREAM_SIZE = 100000
web_traffic_stream = stream(STREAM_SIZE)

## Data structures

In [203]:
import mmh3
import random
import sys
import math
import statistics as s
# number of hash functions (63 bit (signed))

def hashElement(element, seed):
    a,b = mmh3.hash64(element, seed, signed=False)
    return a

def buildHyperLogLog(stream, streamSize, rnd):
    m = 64;
    M = [0]*m
    seed = 42
    if(rnd):
        seed = random.randint(0,sys.maxsize/2)
    #Hash every element in the stream
    for e in stream:
        hE = hashElement(e, seed)
        bitstring = hE
        bitFullLength = format(bitstring, '#066b')[2:]
        size_upper_part = math.floor(math.log(m,2))
        size_lower_part = m - size_upper_part
        upperpart = bitstring >> size_lower_part
        lowerpart = bitstring & ((2 ** size_lower_part) -1)
        # p is the position of the left most bit
        p = 64 - int(math.log(lowerpart,2))
        # j is the integer representation of the upperpart, we do not add one as python has 0 indexed 
        j = upperpart 
        #if(p>M[j]): print(str(p)); print(bin(upperpart))
        #if(j>=63):
            #print("upper: " + str(upperpart))
            #print("lower: " + bin(lowerpart))
            #print("hash:  " + bin(hE))
        M[j] = max(M[j],p)
    return M

def getHarmonicMean(dataStucture):
    return s.harmonic_mean(dataStucture)

#alpha is the constant adjust to correct for bias. It's value is defined on Wiki: 
#https://en.wikipedia.org/wiki/HyperLogLog#Operations
alpha = 0.709

web_traffic_stream = stream(STREAM_SIZE)
ds = buildHyperLogLog(web_traffic_stream, STREAM_SIZE, True)
print(ds)
hm = getHarmonicMean(ds)

E = alpha * (m) * hm
print(E)


[9, 10, 9, 14, 10, 12, 15, 11, 9, 9, 12, 10, 10, 10, 16, 13, 11, 12, 18, 10, 9, 11, 9, 15, 8, 11, 11, 10, 10, 11, 11, 9, 11, 13, 12, 13, 11, 10, 12, 11, 10, 11, 9, 14, 13, 9, 11, 11, 13, 12, 11, 10, 13, 10, 10, 9, 12, 10, 15, 13, 11, 11, 12, 10]
495.8535104935064


In [292]:
import mmh3
## CountMin sketch
def chooseDHashFunctions(d):
    hashFunctions = [0]*d
    for i in range(len(hashFunctions)):
        hashFunctions[i] = random.randint(0,64)
    return hashFunctions

def initializeArrayM(d,w):
    M = []
    for i in range(len(d)):
        #print(i)
        M.append([d[i]])
        for j in range(w):
            M[i].append([0])
    return M

def hashElement(elem, seed):
    if(seed == None): 
        seed = 42
    h = mmh3.hash(elem, seed, signed=False)
    return h

def setDSWithNewElem(M, elem, w):
    i = 0
    for hF in M:
        h = hashElement(elem,hF[0])
        index = h % w
        print(hF[0])
        print("index " + str(index))
        print("value " + str(M[i][index]))
        M[i][index] = [1]
        i += 1
    return M

hF = (chooseDHashFunctions(2))
w = 6
M = initializeArrayM(hF,w)

setDSWithNewElem(M, 'Hello',w)

45
index 1
value[0]
56
index 0
value56


[[45, [1], [0], [0], [0], [0], [0]], [[1], [0], [0], [0], [0], [0], [0]]]

## Analysis

In [182]:
#print("hash: "+str(hE))

hE = hashElement("hello", random.randint(0,sys.maxsize/2))
bitstring = hE
bitstringSliced = bin(bitstring)[2:]
long_bitstring = format(hE, '#064b')
size_upper_part = math.floor(math.log(bitstring.bit_length(),2))
size_lower_part = 64 - size_upper_part
upperpart = (bitstring >> size_lower_part)
long_upper = format(upperpart, '#08b')
lowerpart = bitstring & ((2 ** size_lower_part) -1)
lUasint = int(long_upper, 2)


print(_bit_length(bitstring))
print("Long org: " + long_bitstring)
print("org len: " + str(len(long_bitstring)))
print("org:    " + bin(bitstring))
print("upper:  " + bin(upperpart))
print("lower:  " + bin(lowerpart))
print(int(long_upper, 2))
print("Position upper: " + str(upperpart.bit_length()))
print("Position lower: " + str(lowerpart.bit_length()))


62
Long org: 0b11011000111110101001110101011111011001110000000111010100111000
org len: 64
org:    0b11011000111110101001110101011111011001110000000111010100111000
upper:  0b110
lower:  0b11000111110101001110101011111011001110000000111010100111000
6
Position upper: 3
Position lower: 59


In [344]:
import numpy as np

def createM(d, w): 
    table = np.zeros([d, w])
    return table

def generateSeeds(d, w):
    seeds = np.random.randint(w, size = d)
    return seeds


def increment(seeds, key, table, d, w):
    for i in range(0, d):
        index = mmh3.hash(key, seeds[i]) % w
        table[i, index] = table[i, index]+1 
    return table

d = 64
w = 64
M = createM(d, w)
seeds = generateSeeds(d,w)
web_traffic_stream = stream(STREAM_SIZE)
for e in web_traffic_stream:
    increment(seeds,e,M,d,w)
    
def estimateResult(seeds, table, key, d, w):
        min_est = 100000
        for i in range(0, d):
            index = mmh3.hash(key, seeds[i]) % w
            if table[i, index] < min_est:
                min_est = table[i, index]
        return min_est
    
web_traffic_stream = stream(STREAM_SIZE)


for e in web_traffic_stream:
    print(estimateResult(seeds, M, e, d,w))


1489.0
1484.0
1505.0
1482.0
1456.0
1487.0
1472.0
1509.0
1487.0
1499.0
1490.0
1508.0
1469.0
1473.0
1475.0
1481.0
1425.0
1457.0
1475.0
1491.0
1491.0
1497.0
1440.0
1459.0
1472.0
1505.0
1456.0
1479.0
1459.0
1469.0
1495.0
1457.0
1493.0
1484.0
1496.0
1457.0
1443.0
1440.0
1445.0
1486.0
1499.0
1447.0
1478.0
1494.0
1472.0
1447.0
1472.0
1477.0
1475.0
1505.0
1493.0
1475.0
1518.0
1472.0
1512.0
1469.0
1477.0
1488.0
1466.0
1469.0
1490.0
1475.0
1460.0
1490.0
1475.0
1460.0
1508.0
1457.0
1477.0
1471.0
1445.0
1502.0
1480.0
1474.0
1489.0
1481.0
1475.0
1505.0
1472.0
1475.0
1463.0
1517.0
1463.0
1501.0
1459.0
1499.0
1503.0
1492.0
1490.0
1460.0
1487.0
1457.0
1443.0
1511.0
1499.0
1469.0
1450.0
1443.0
1463.0
1490.0
1484.0
1499.0
1481.0
1466.0
1477.0
1487.0
1487.0
1487.0
1468.0
1472.0
1443.0
1475.0
1459.0
1505.0
1475.0
1484.0
1463.0
1472.0
1483.0
1466.0
1507.0
1494.0
1471.0
1459.0
1474.0
1484.0
1462.0
1489.0
1500.0
1460.0
1503.0
1440.0
1505.0
1472.0
1484.0
1520.0
1503.0
1512.0
1501.0
1443.0
1480.0
1456.0
1506.0

1499.0
1459.0
1483.0
1475.0
1481.0
1494.0
1501.0
1500.0
1443.0
1483.0
1509.0
1483.0
1487.0
1466.0
1472.0
1492.0
1462.0
1477.0
1485.0
1457.0
1475.0
1479.0
1440.0
1456.0
1475.0
1498.0
1481.0
1445.0
1475.0
1474.0
1479.0
1476.0
1473.0
1492.0
1462.0
1475.0
1508.0
1481.0
1460.0
1488.0
1490.0
1477.0
1477.0
1440.0
1476.0
1471.0
1495.0
1508.0
1456.0
1472.0
1469.0
1499.0
1425.0
1459.0
1440.0
1493.0
1459.0
1487.0
1473.0
1505.0
1495.0
1494.0
1508.0
1459.0
1501.0
1485.0
1495.0
1477.0
1445.0
1510.0
1475.0
1471.0
1480.0
1472.0
1459.0
1492.0
1456.0
1480.0
1491.0
1487.0
1480.0
1475.0
1472.0
1481.0
1479.0
1488.0
1504.0
1485.0
1503.0
1456.0
1508.0
1443.0
1469.0
1497.0
1484.0
1499.0
1456.0
1481.0
1475.0
1456.0
1496.0
1503.0
1459.0
1500.0
1485.0
1472.0
1460.0
1490.0
1479.0
1471.0
1472.0
1486.0
1460.0
1482.0
1477.0
1475.0
1498.0
1499.0
1477.0
1443.0
1459.0
1511.0
1475.0
1477.0
1482.0
1485.0
1490.0
1475.0
1478.0
1527.0
1485.0
1528.0
1487.0
1480.0
1462.0
1497.0
1475.0
1447.0
1494.0
1480.0
1499.0
1460.0
1473.0

1475.0
1475.0
1500.0
1463.0
1499.0
1443.0
1484.0
1468.0
1477.0
1443.0
1503.0
1490.0
1473.0
1443.0
1487.0
1477.0
1475.0
1457.0
1459.0
1507.0
1505.0
1521.0
1491.0
1468.0
1488.0
1487.0
1457.0
1440.0
1445.0
1501.0
1480.0
1490.0
1471.0
1502.0
1512.0
1485.0
1457.0
1501.0
1469.0
1493.0
1468.0
1495.0
1475.0
1475.0
1488.0
1473.0
1407.0
1505.0
1475.0
1484.0
1503.0
1489.0
1443.0
1498.0
1475.0
1425.0
1494.0
1460.0
1485.0
1489.0
1499.0
1468.0
1501.0
1487.0
1503.0
1463.0
1479.0
1492.0
1425.0
1459.0
1492.0
1456.0
1477.0
1511.0
1443.0
1440.0
1496.0
1495.0
1505.0
1511.0
1472.0
1466.0
1460.0
1499.0
1484.0
1477.0
1498.0
1456.0
1506.0
1508.0
1516.0
1468.0
1471.0
1456.0
1443.0
1475.0
1514.0
1477.0
1472.0
1471.0
1477.0
1485.0
1443.0
1497.0
1407.0
1480.0
1447.0
1466.0
1499.0
1505.0
1482.0
1479.0
1511.0
1479.0
1475.0
1456.0
1491.0
1484.0
1501.0
1496.0
1457.0
1496.0
1502.0
1443.0
1508.0
1485.0
1498.0
1480.0
1490.0
1505.0
1502.0
1471.0
1456.0
1506.0
1471.0
1443.0
1477.0
1482.0
1456.0
1480.0
1489.0
1480.0
1485.0

1475.0
1471.0
1477.0
1511.0
1500.0
1487.0
1456.0
1459.0
1497.0
1495.0
1495.0
1505.0
1472.0
1471.0
1496.0
1482.0
1479.0
1474.0
1498.0
1492.0
1443.0
1486.0
1460.0
1440.0
1490.0
1485.0
1481.0
1487.0
1443.0
1520.0
1460.0
1511.0
1487.0
1487.0
1492.0
1479.0
1481.0
1493.0
1456.0
1497.0
1500.0
1448.0
1462.0
1443.0
1485.0
1440.0
1481.0
1457.0
1475.0
1462.0
1472.0
1471.0
1445.0
1503.0
1460.0
1447.0
1500.0
1477.0
1505.0
1484.0
1493.0
1481.0
1475.0
1443.0
1459.0
1459.0
1475.0
1477.0
1460.0
1477.0
1509.0
1487.0
1480.0
1487.0
1457.0
1493.0
1468.0
1456.0
1425.0
1487.0
1482.0
1487.0
1505.0
1499.0
1462.0
1443.0
1484.0
1469.0
1484.0
1514.0
1475.0
1501.0
1462.0
1509.0
1476.0
1472.0
1484.0
1476.0
1486.0
1407.0
1472.0
1497.0
1477.0
1511.0
1471.0
1504.0
1488.0
1457.0
1457.0
1459.0
1479.0
1487.0
1440.0
1495.0
1477.0
1460.0
1440.0
1492.0
1450.0
1466.0
1492.0
1475.0
1480.0
1425.0
1487.0
1425.0
1498.0
1501.0
1448.0
1440.0
1456.0
1460.0
1496.0
1506.0
1492.0
1500.0
1508.0
1483.0
1481.0
1484.0
1480.0
1472.0
1517.0

1508.0
1492.0
1504.0
1477.0
1481.0
1479.0
1512.0
1479.0
1495.0
1475.0
1443.0
1517.0
1509.0
1493.0
1460.0
1460.0
1486.0
1450.0
1480.0
1459.0
1477.0
1460.0
1475.0
1460.0
1460.0
1485.0
1457.0
1511.0
1502.0
1496.0
1482.0
1445.0
1480.0
1459.0
1475.0
1407.0
1471.0
1460.0
1480.0
1479.0
1473.0
1471.0
1425.0
1460.0
1480.0
1491.0
1475.0
1450.0
1493.0
1511.0
1456.0
1487.0
1486.0
1471.0
1481.0
1490.0
1447.0
1487.0
1450.0
1507.0
1460.0
1491.0
1440.0
1456.0
1499.0
1469.0
1472.0
1472.0
1476.0
1489.0
1488.0
1511.0
1485.0
1445.0
1496.0
1502.0
1500.0
1484.0
1472.0
1450.0
1475.0
1505.0
1483.0
1460.0
1463.0
1443.0
1487.0
1515.0
1425.0
1448.0
1484.0
1489.0
1456.0
1492.0
1443.0
1475.0
1518.0
1459.0
1443.0
1501.0
1459.0
1483.0
1495.0
1505.0
1487.0
1477.0
1506.0
1490.0
1505.0
1487.0
1459.0
1498.0
1484.0
1488.0
1443.0
1475.0
1469.0
1495.0
1457.0
1479.0
1492.0
1469.0
1498.0
1447.0
1506.0
1459.0
1486.0
1505.0
1497.0
1478.0
1466.0
1493.0
1498.0
1474.0
1485.0
1507.0
1443.0
1485.0
1497.0
1499.0
1448.0
1488.0
1472.0

...

1479.0
1490.0
1477.0
1469.0
1473.0
1487.0
1456.0
1480.0
1475.0
1476.0
1495.0
1481.0
1459.0
1501.0
1486.0
1407.0
1472.0
1500.0
1448.0
1517.0
1472.0
1477.0
1480.0
1457.0
1492.0
1495.0
1466.0
1489.0
1466.0
1501.0
1509.0
1468.0
1485.0
1478.0
1443.0
1484.0
1443.0
1475.0
1463.0
1463.0
1476.0
1478.0
1440.0
1475.0
1478.0
1492.0
1425.0
1496.0
1508.0
1475.0
1469.0
1477.0
1490.0
1450.0
1487.0
1484.0
1490.0
1447.0
1472.0
1489.0
1474.0
1469.0
1501.0
1457.0
1443.0
1443.0
1495.0
1506.0
1456.0
1463.0
1492.0
1475.0
1503.0
1473.0
1447.0
1475.0
1490.0
1456.0
1485.0
1479.0
1493.0
1479.0
1476.0
1506.0
1463.0
1485.0
1486.0
1484.0
1480.0
1486.0
1491.0
1506.0
1480.0
1466.0
1488.0
1477.0
1494.0
1472.0
1505.0
1445.0
1508.0
1487.0
1484.0
1475.0
1510.0
1487.0
1494.0
1498.0
1479.0
1440.0
1493.0
1456.0
1487.0
1456.0
1472.0
1487.0
1476.0
1480.0
1480.0
1498.0
1500.0
1496.0
1466.0
1447.0
1506.0
1457.0
1479.0
1492.0
1443.0
1477.0
1507.0
1499.0
1480.0
1443.0
1447.0
1456.0
1456.0
1474.0
1407.0
1500.0
1505.0
1493.0
1510.0

1485.0
1487.0
1503.0
1440.0
1456.0
1479.0
1466.0
1443.0
1475.0
1503.0
1443.0
1447.0
1510.0
1457.0
1443.0
1475.0
1475.0
1460.0
1425.0
1469.0
1472.0
1487.0
1407.0
1498.0
1481.0
1486.0
1477.0
1480.0
1443.0
1475.0
1425.0
1407.0
1487.0
1477.0
1407.0
1510.0
1473.0
1496.0
1472.0
1485.0
1487.0
1498.0
1503.0
1490.0
1472.0
1510.0
1485.0
1459.0
1499.0
1492.0
1477.0
1484.0
1480.0
1456.0
1463.0
1487.0
1495.0
1474.0
1450.0
1479.0
1443.0
1480.0
1477.0
1468.0
1499.0
1508.0
1407.0
1473.0
1490.0
1456.0
1456.0
1505.0
1463.0
1477.0
1460.0
1483.0
1443.0
1493.0
1443.0
1489.0
1425.0
1443.0
1505.0
1479.0
1475.0
1477.0
1469.0
1487.0
1466.0
1492.0
1487.0
1500.0
1501.0
1507.0
1443.0
1485.0
1489.0
1475.0
1475.0
1445.0
1508.0
1472.0
1462.0
1407.0
1507.0
1477.0
1472.0
1492.0
1443.0
1477.0
1447.0
1407.0
1501.0
1487.0
1457.0
1469.0
1443.0
1496.0
1469.0
1445.0
1478.0
1481.0
1474.0
1500.0
1495.0
1492.0
1448.0
1472.0
1500.0
1505.0
1508.0
1488.0
1501.0
1501.0
1460.0
1487.0
1491.0
1484.0
1471.0
1469.0
1472.0
1482.0
1505.0

1511.0
1443.0
1503.0
1487.0
1469.0
1475.0
1514.0
1484.0
1425.0
1479.0
1475.0
1472.0
1457.0
1508.0
1476.0
1443.0
1500.0
1507.0
1493.0
1443.0
1481.0
1494.0
1472.0
1485.0
1456.0
1515.0
1459.0
1495.0
1475.0
1479.0
1489.0
1489.0
1447.0
1450.0
1494.0
1472.0
1466.0
1496.0
1459.0
1447.0
1463.0
1489.0
1477.0
1487.0
1407.0
1487.0
1511.0
1501.0
1479.0
1475.0
1505.0
1480.0
1475.0
1485.0
1457.0
1468.0
1457.0
1476.0
1511.0
1459.0
1460.0
1484.0
1480.0
1507.0
1457.0
1478.0
1472.0
1508.0
1484.0
1482.0
1471.0
1480.0
1407.0
1500.0
1468.0
1472.0
1489.0
1475.0
1478.0
1456.0
1498.0
1485.0
1487.0
1477.0
1498.0
1477.0
1469.0
1484.0
1440.0
1448.0
1513.0
1466.0
1501.0
1481.0
1443.0
1481.0
1475.0
1466.0
1479.0
1498.0
1448.0
1481.0
1475.0
1503.0
1475.0
1477.0
1473.0
1450.0
1482.0
1475.0
1477.0
1481.0
1514.0
1460.0
1425.0
1469.0
1498.0
1425.0
1492.0
1479.0
1495.0
1476.0
1487.0
1488.0
1490.0
1477.0
1466.0
1471.0
1512.0
1459.0
1425.0
1407.0
1463.0
1488.0
1457.0
1443.0
1484.0
1501.0
1472.0
1480.0
1471.0
1460.0
1471.0

1443.0
1460.0
1473.0
1425.0
1481.0
1492.0
1490.0
1479.0
1471.0
1492.0
1466.0
1492.0
1495.0
1480.0
1471.0
1499.0
1472.0
1472.0
1487.0
1484.0
1472.0
1476.0
1462.0
1506.0
1508.0
1480.0
1497.0
1469.0
1450.0
1487.0
1483.0
1495.0
1490.0
1468.0
1425.0
1469.0
1460.0
1472.0
1507.0
1474.0
1506.0
1445.0
1460.0
1472.0
1443.0
1477.0
1459.0
1499.0
1466.0
1486.0
1492.0
1460.0
1456.0
1494.0
1477.0
1462.0
1469.0
1473.0
1522.0
1475.0
1466.0
1450.0
1459.0
1515.0
1440.0
1483.0
1466.0
1443.0
1457.0
1475.0
1425.0
1488.0
1460.0
1457.0
1475.0
1457.0
1524.0
1472.0
1485.0
1487.0
1468.0
1496.0
1474.0
1513.0
1484.0
1477.0
1510.0
1485.0
1484.0
1499.0
1477.0
1479.0
1471.0
1502.0
1501.0
1443.0
1499.0
1500.0
1487.0
1471.0
1499.0
1496.0
1511.0
1487.0
1484.0
1425.0
1457.0
1480.0
1457.0
1445.0
1481.0
1468.0
1459.0
1506.0
1509.0
1479.0
1500.0
1425.0
1494.0
1491.0
1487.0
1481.0
1473.0
1485.0
1491.0
1504.0
1482.0
1483.0
1509.0
1469.0
1477.0
1484.0
1456.0
1457.0
1489.0
1469.0
1471.0
1492.0
1456.0
1496.0
1450.0
1494.0
1479.0

1478.0
1463.0
1503.0
1445.0
1505.0
1472.0
1493.0
1499.0
1447.0
1480.0
1483.0
1475.0
1482.0
1477.0
1480.0
1475.0
1443.0
1504.0
1407.0
1504.0
1493.0
1502.0
1476.0
1479.0
1445.0
1507.0
1491.0
1440.0
1477.0
1483.0
1479.0
1497.0
1482.0
1502.0
1488.0
1456.0
1503.0
1475.0
1487.0
1469.0
1460.0
1495.0
1490.0
1481.0
1475.0
1485.0
1462.0
1475.0
1479.0
1485.0
1471.0
1510.0
1499.0
1487.0
1499.0
1440.0
1462.0
1483.0
1479.0
1447.0
1443.0
1503.0
1462.0
1425.0
1471.0
1471.0
1495.0
1508.0
1501.0
1466.0
1491.0
1510.0
1487.0
1443.0
1503.0
1508.0
1492.0
1466.0
1482.0
1482.0
1484.0
1471.0
1456.0
1456.0
1475.0
1479.0
1497.0
1447.0
1471.0
1487.0
1440.0
1484.0
1440.0
1407.0
1502.0
1443.0
1472.0
1507.0
1460.0
1472.0
1503.0
1472.0
1501.0
1447.0
1499.0
1443.0
1443.0
1466.0
1484.0
1478.0
1468.0
1498.0
1480.0
1469.0
1450.0
1495.0
1460.0
1498.0
1443.0
1482.0
1459.0
1478.0
1480.0
1491.0
1448.0
1516.0
1479.0
1488.0
1487.0
1487.0
1484.0
1447.0
1474.0
1478.0
1445.0
1490.0
1486.0
1501.0
1472.0
1477.0
1503.0
1490.0
1520.0

In [346]:
import numpy as np
d = 64
w = 64

def init_countMinSketchIp(d, w):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)
    seed = generateSeeds(d,w)

    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        M = increment(seed,ip,M,d,w)
    return M
def init_countMinSketchDomain(d, w):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)
    seed = generateSeeds(d,w)

    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        increment(seed,domain,M,d,w)
        
def countIp_countMinSketch(seed, M, targetIp, d, w):
    print(estimateResult(seed, M, targetIp, d,w))

def countDomain_countMinSketch(seed, M, targetDomain, d, w):
    print(estimateResult(seed, M, targetDomain, d,w))

def countAllDomain(seed, M, targetDomain, d, w):
    web_traffic_stream = stream(STREAM_SIZE)
    for e in web_traffic_stream:
        print(estimateResult(seed, M, targetDomain, d,w))
        
M = init_countMinSketchIp(d,w)
seed = generateSeeds(d,w)
web_traffic_stream = stream(STREAM_SIZE)
e = next(web_traffic_stream).split() 
targetIp = e[0]
targetDomain = e[1]

countIp_countMinSketch(seed, M, targetIp, d, w)
countDomain_countMinSketch(seed, M, targetDomain, d, w)

e2 = next(web_traffic_stream).split() 
targetIp2 = e2[0]
targetDomain2 = e2[1]

countIp_countMinSketch(seed, M, targetIp2, d, w)
countDomain_countMinSketch(seed, M, targetDomain2, d, w)



1490.0
1464.0
1479.0
1463.0
